In [2]:
import multiprocessing as mp
from multiprocessing import Manager
import time
import logging
import queue

from logging.handlers import QueueHandler, QueueListener

from motion_control_w_logging import MotionControl, ComputerVision, setup_logging, log_listener

class RobotController:
    def __init__(self):
        self.manager = Manager()
        self.shared_data = {
            'running': self.manager.Value('b', True),
            'position': self.manager.dict({'x': 0, 'y': 0}),
            'motion_state': self.manager.Value('s', 'STOPPED'),
            'detected_objects': self.manager.list()
        }
        
        self.log_queue = self.manager.Queue()
        self.command_queue = self.manager.Queue()
        self.status_queue = self.manager.Queue()
        self.vision_queue = self.manager.Queue()
        
        self.motion_control = MotionControl(self.shared_data, self.command_queue, self.status_queue, self.log_queue)
        self.computer_vision = ComputerVision(self.shared_data, self.vision_queue, self.log_queue)
        
        self.listener = log_listener(self.log_queue)
        setup_logging(self.log_queue)
        self.logger = logging.getLogger(f'{__name__}.RobotController')

    def start(self):
        self.motion_control.start()
        self.computer_vision.start()

    def stop(self):
        self.shared_data['running'] = False
        self.motion_control.join()
        self.computer_vision.join()
        self.listener.stop()

    def run(self):
        self.start()
        try:
            while True:
                # Check vision data
                try:
                    vision_data = self.vision_queue.get_nowait()
                    if vision_data['type'] == 'obstacle' and vision_data['distance'] < 2.0:
                        self.logger.warning("Obstacle detected nearby! Stopping motion.")
                        self.command_queue.put('STOP')
                except Exception:
                    pass

                # Check motion status
                try:
                    motion_status = self.status_queue.get_nowait()
                    if motion_status['motion_state'] == 'STOPPED':
                        self.logger.info("Robot is stopped. Checking if path is clear...")
                        # Check if path is clear (you'd implement more sophisticated logic here)
                        if len(self.shared_data['detected_objects']) == 0:
                            self.logger.info("Path is clear. Resuming motion.")
                            self.command_queue.put('MOVE')
                except Exception:
                    pass

                # Periodic status update
                if time.time() % 5 < 0.1:  # Approximately every 5 seconds
                    self.logger.info(f"Current position: {dict(self.shared_data['position'])}")
                    self.logger.info(f"Motion state: {self.shared_data['motion_state']}")
                    self.logger.info(f"Detected objects: {list(self.shared_data['detected_objects'])}")

                time.sleep(0.1)

        except KeyboardInterrupt:
            self.logger.info("Stopping robot...")
            self.stop()

if __name__ == "__main__":
    controller = RobotController()
    controller.run()

Process MotionControl-5:
Process ComputerVision-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/patrick/miniconda3/envs/random-tools/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/patrick/Documents/Projects/ECE4191-E5-tennibot/notebooks/scratchspace/patricks_random/multithread_playground/motion_control_w_logging.py", line 55, in run
    time.sleep(0.1)
KeyboardInterrupt
Stopping robot...
  File "/Users/patrick/miniconda3/envs/random-tools/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/patrick/Documents/Projects/ECE4191-E5-tennibot/notebooks/scratchspace/patricks_random/multithread_playground/motion_control_w_logging.py", line 77, in run
    time.sleep(0.5)
KeyboardInterrupt
